In [1]:
from agential.agents.expel.agent import ExpeL
from agential.agents.reflexion.agent import ReflexionReAct
from agential.utils.docstore import DocstoreExplorer
from agential.core.llm import LLM

import tiktoken

from agential.core.fewshots.hotpotqa import (
    HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
)
from agential.core.fewshots.fever import (
    FEVER_FEWSHOT_EXAMPLES_REACT,
)
from agential.core.fewshots.triviaqa import (
    TRIVIAQA_FEWSHOT_EXAMPLES_REACT,
)
from agential.core.fewshots.ambignq import (
    AMBIGNQ_FEWSHOT_EXAMPLES_REACT,
)
from agential.core.fewshots.gsm8k import (
    GSM8K_FEWSHOT_EXAMPLES_REACT
)
from agential.core.fewshots.svamp import (
    SVAMP_FEWSHOT_EXAMPLES_REACT
)
from agential.core.fewshots.tabmwp import (
    TABMWP_FEWSHOT_EXAMPLES_REACT
)
from agential.core.fewshots.humaneval import (
    HUMANEVAL_FEWSHOT_EXAMPLES_REACT
)
from agential.core.fewshots.mbpp import (
    MBPP_FEWSHOT_EXAMPLES_REACT
)
from agential.agents.reflexion.prompts import (
    AMBIGNQ_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    FEVER_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    GSM8K_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    HUMANEVAL_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    MBPP_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    REFLEXION_REACT_REFLECT_INSTRUCTION_AMBIGNQ,
    REFLEXION_REACT_REFLECT_INSTRUCTION_FEVER,
    REFLEXION_REACT_REFLECT_INSTRUCTION_GSM8K,
    REFLEXION_REACT_REFLECT_INSTRUCTION_HOTPOTQA,
    REFLEXION_REACT_REFLECT_INSTRUCTION_HUMANEVAL,
    REFLEXION_REACT_REFLECT_INSTRUCTION_MBPP,
    REFLEXION_REACT_REFLECT_INSTRUCTION_SVAMP,
    REFLEXION_REACT_REFLECT_INSTRUCTION_TABMWP,
    REFLEXION_REACT_REFLECT_INSTRUCTION_TRIVIAQA,
    SVAMP_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    TABMWP_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    TRIVIAQA_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
)
from agential.agents.expel.prompts import (
    EXPEL_REFLEXION_REACT_INSTRUCTION_AMBIGNQ,
    EXPEL_REFLEXION_REACT_INSTRUCTION_FEVER,
    EXPEL_REFLEXION_REACT_INSTRUCTION_GSM8K,
    EXPEL_REFLEXION_REACT_INSTRUCTION_HOTPOTQA,
    EXPEL_REFLEXION_REACT_INSTRUCTION_HUMANEVAL,
    EXPEL_REFLEXION_REACT_INSTRUCTION_MBPP,
    EXPEL_REFLEXION_REACT_INSTRUCTION_SVAMP,
    EXPEL_REFLEXION_REACT_INSTRUCTION_TABMWP,
    EXPEL_REFLEXION_REACT_INSTRUCTION_TRIVIAQA,
)
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

from langchain_community.docstore.wikipedia import Wikipedia
from agential.agents.reflexion.reflect import (
    ReflexionReActReflector,
)
from agential.agents.expel.memory import (
    ExpeLExperienceMemory,
    ExpeLInsightMemory
)
import warnings
warnings.filterwarnings('ignore')

import dotenv
dotenv.load_dotenv()

llm = LLM("gpt-3.5-turbo")

/opt/anaconda3/envs/agential/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/agential/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# QA

### HotpotQA

In [2]:
question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"
key = "Gesellschaft mit beschränkter Haftung"

benchmark = "hotpotqa"

reflexion_react_agent = ReflexionReAct(
    llm=llm,
    benchmark=benchmark,
    reflector=ReflexionReActReflector(llm=llm),
    # kwargs.
    max_reflections=3,
    max_trials=1,
    max_steps=6,
    max_tokens=5000,
    enc=tiktoken.encoding_for_model("gpt-3.5-turbo"),
    docstore=DocstoreExplorer(Wikipedia()),
)
agent = ExpeL(
    llm=llm,
    benchmark=benchmark,
    reflexion_react_agent=reflexion_react_agent,
    experience_memory=ExpeLExperienceMemory(
        experiences=[],
        strategy="task",
        embedder=HuggingFaceEmbeddings(),
        encoder=tiktoken.encoding_for_model("gpt-3.5-turbo")
    ),
    insight_memory=ExpeLInsightMemory(
        insights=[],
        max_num_insights=20,
        leeway=5
    ),
    # kwargs.
    reflexion_react_strategy_kwargs={"max_steps": 7, "max_trials": 3},
    success_batch_size=8,
    extract_init_insights=True,
)

out = agent.generate(
    question=question,
    key=key,
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    prompt=EXPEL_REFLEXION_REACT_INSTRUCTION_HOTPOTQA,
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    reflect_prompt=REFLEXION_REACT_REFLECT_INSTRUCTION_HOTPOTQA,
    reflect_strategy="reflexion",
    additional_keys={},
    reflect_additional_keys={},
    use_dynamic_examples=True,
    extract_insights=True,
    patience=3,
    k_docs=24,
    num_fewshots=6,
    max_fewshot_tokens=1500,
    reranker_strategy=None,
    reset=False,
)

In [3]:
out

ExpeLOutput(answer='unable to provide an answer', total_prompt_tokens=25224, total_completion_tokens=1988, total_tokens=27212, total_prompt_cost=0.012611999999999998, total_completion_cost=0.0029820000000000003, total_cost=0.015593999999999998, total_prompt_time=21.89635705947876, total_time=26.87044668197632, additional_info=ExpeLGenerateOutput(examples='Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?\nThought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.\nAction 1: Search[Colorado orogeny]\nObservation 1: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.\nThought 2: It does not mention the eastern sector. So I need to look up eastern sector.\nAction 2: Lookup[eastern sector]\nObservation 2: (Result 1 / 1) The eastern sector extends into the High Plains and 

### FEVER

In [ ]:
question = "Nikolaj Coster-Waldau worked with the Fox Broadcasting Company."
key = "REFUTES"

benchmark = "fever"

reflexion_react_agent = ReflexionReAct(
    llm=llm,
    benchmark=benchmark,
    reflector=ReflexionReActReflector(llm=llm),
    # kwargs.
    max_reflections=3,
    max_trials=1,
    max_steps=6,
    max_tokens=5000,
    enc=tiktoken.encoding_for_model("gpt-3.5-turbo"),
    docstore=DocstoreExplorer(Wikipedia()),
)
agent = ExpeL(
    llm=llm,
    benchmark=benchmark,
    reflexion_react_agent=reflexion_react_agent,
    experience_memory=ExpeLExperienceMemory(
        experiences=[],
        strategy="task",
        embedder=HuggingFaceEmbeddings(),
        encoder=tiktoken.encoding_for_model("gpt-3.5-turbo")
    ),
    insight_memory=ExpeLInsightMemory(
        insights=[],
        max_num_insights=20,
        leeway=5
    ),
    # kwargs.
    reflexion_react_strategy_kwargs={"max_steps": 7, "max_trials": 3},
    success_batch_size=8,
    extract_init_insights=True,
)

out = agent.generate(
    question=question,
    key=key,
    examples=FEVER_FEWSHOT_EXAMPLES_REACT,
    prompt=EXPEL_REFLEXION_REACT_INSTRUCTION_FEVER,
    reflect_examples=FEVER_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    reflect_prompt=REFLEXION_REACT_REFLECT_INSTRUCTION_FEVER,
    reflect_strategy="reflexion",
    additional_keys={},
    reflect_additional_keys={},
    use_dynamic_examples=True,
    extract_insights=True,
    patience=3,
    k_docs=24,
    num_fewshots=6,
    max_fewshot_tokens=1500,
    reranker_strategy=None,
    reset=False,
)


In [ ]:
out

### AmbigNQ

In [ ]:
question = "When did the simpsons first air on television?"
key = "1989"

benchmark = "ambignq"

reflexion_react_agent = ReflexionReAct(
    llm=llm,
    benchmark=benchmark,
    reflector=ReflexionReActReflector(llm=llm),
    # kwargs.
    max_reflections=3,
    max_trials=1,
    max_steps=6,
    max_tokens=5000,
    enc=tiktoken.encoding_for_model("gpt-3.5-turbo"),
    docstore=DocstoreExplorer(Wikipedia()),
)
agent = ExpeL(
    llm=llm,
    benchmark=benchmark,
    reflexion_react_agent=reflexion_react_agent,
    experience_memory=ExpeLExperienceMemory(
        experiences=[],
        strategy="task",
        embedder=HuggingFaceEmbeddings(),
        encoder=tiktoken.encoding_for_model("gpt-3.5-turbo")
    ),
    insight_memory=ExpeLInsightMemory(
        insights=[],
        max_num_insights=20,
        leeway=5
    ),
    # kwargs.
    reflexion_react_strategy_kwargs={"max_steps": 7, "max_trials": 3},
    success_batch_size=8,
    extract_init_insights=True,
)

out = agent.generate(
    question=question,
    key=key,
    examples=AMBIGNQ_FEWSHOT_EXAMPLES_REACT,
    prompt=EXPEL_REFLEXION_REACT_INSTRUCTION_AMBIGNQ,
    reflect_examples=AMBIGNQ_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    reflect_prompt=REFLEXION_REACT_REFLECT_INSTRUCTION_AMBIGNQ,
    reflect_strategy="reflexion",
    additional_keys={},
    reflect_additional_keys={},
    use_dynamic_examples=True,
    extract_insights=True,
    patience=3,
    k_docs=24,
    num_fewshots=6,
    max_fewshot_tokens=1500,
    reranker_strategy=None,
    reset=False,
)


In [ ]:
out

### TriviaQA

In [ ]:
question = "Which American-born Sinclair won the Nobel Prize for Literature in 1930?"
key = "Sinclair Lewis"

benchmark = "triviaqa"

reflexion_react_agent = ReflexionReAct(
    llm=llm,
    benchmark=benchmark,
    reflector=ReflexionReActReflector(llm=llm),
    # kwargs.
    max_reflections=3,
    max_trials=1,
    max_steps=6,
    max_tokens=5000,
    enc=tiktoken.encoding_for_model("gpt-3.5-turbo"),
    docstore=DocstoreExplorer(Wikipedia()),
)
agent = ExpeL(
    llm=llm,
    benchmark=benchmark,
    reflexion_react_agent=reflexion_react_agent,
    experience_memory=ExpeLExperienceMemory(
        experiences=[],
        strategy="task",
        embedder=HuggingFaceEmbeddings(),
        encoder=tiktoken.encoding_for_model("gpt-3.5-turbo")
    ),
    insight_memory=ExpeLInsightMemory(
        insights=[],
        max_num_insights=20,
        leeway=5
    ),
    # kwargs.
    reflexion_react_strategy_kwargs={"max_steps": 7, "max_trials": 3},
    success_batch_size=8,
    extract_init_insights=True,
)

out = agent.generate(
    question=question,
    key=key,
    examples=TRIVIAQA_FEWSHOT_EXAMPLES_REACT,
    prompt=EXPEL_REFLEXION_REACT_INSTRUCTION_TRIVIAQA,
    reflect_examples=TRIVIAQA_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    reflect_prompt=REFLEXION_REACT_REFLECT_INSTRUCTION_TRIVIAQA,
    reflect_strategy="reflexion",
    additional_keys={},
    reflect_additional_keys={},
    use_dynamic_examples=True,
    extract_insights=True,
    patience=3,
    k_docs=24,
    num_fewshots=6,
    max_fewshot_tokens=1500,
    reranker_strategy=None,
    reset=False,  
)


In [ ]:
out

# Math

### GSM8K

In [2]:
question = "Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with 4933828. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"
key = "-9867630"

benchmark = "gsm8k"

reflexion_react_agent = ReflexionReAct(
    llm=llm,
    benchmark=benchmark,
    reflector=ReflexionReActReflector(llm=llm),
    # kwargs.
    max_reflections=3,
    max_trials=1,
    max_steps=6,
    max_tokens=5000,
    enc=tiktoken.encoding_for_model("gpt-3.5-turbo"),
)
agent = ExpeL(
    llm=llm,
    benchmark=benchmark,
    reflexion_react_agent=reflexion_react_agent,
    experience_memory=ExpeLExperienceMemory(
        experiences=[],
        strategy="task",
        embedder=HuggingFaceEmbeddings(),
        encoder=tiktoken.encoding_for_model("gpt-3.5-turbo")
    ),
    insight_memory=ExpeLInsightMemory(
        insights=[],
        max_num_insights=20,
        leeway=5
    ),
    # kwargs.
    reflexion_react_strategy_kwargs={"max_steps": 7, "max_trials": 3},
    success_batch_size=8,
    extract_init_insights=True,
)

out = agent.generate(
    question=question,
    key=key,
    examples=GSM8K_FEWSHOT_EXAMPLES_REACT,
    prompt=EXPEL_REFLEXION_REACT_INSTRUCTION_GSM8K,
    reflect_examples=GSM8K_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    reflect_prompt=REFLEXION_REACT_REFLECT_INSTRUCTION_GSM8K,
    reflect_strategy="reflexion",
    additional_keys={},
    reflect_additional_keys={},
    use_dynamic_examples=True,
    extract_insights=True,
    patience=3,
    k_docs=24,
    num_fewshots=6,
    max_fewshot_tokens=1500,
    reranker_strategy=None,
    reset=False,
)


In [ ]:
out

### SVAMP

In [ ]:
question = "There are 87 oranges and 290 bananas in Philip's collection. If the bananas are organized into 2 groups and oranges are organized into 93 groups. How big is each group of bananas?"
key = "145"


benchmark = "svamp"

reflexion_react_agent = ReflexionReAct(
    llm=llm,
    benchmark=benchmark,
    reflector=ReflexionReActReflector(llm=llm),
    # kwargs.
    max_reflections=3,
    max_trials=1,
    max_steps=6,
    max_tokens=5000,
    enc=tiktoken.encoding_for_model("gpt-3.5-turbo"),
)
agent = ExpeL(
    llm=llm,
    benchmark=benchmark,
    reflexion_react_agent=reflexion_react_agent,
    experience_memory=ExpeLExperienceMemory(
        experiences=[],
        strategy="task",
        embedder=HuggingFaceEmbeddings(),
        encoder=tiktoken.encoding_for_model("gpt-3.5-turbo")
    ),
    insight_memory=ExpeLInsightMemory(
        insights=[],
        max_num_insights=20,
        leeway=5
    ),
    # kwargs.
    reflexion_react_strategy_kwargs={"max_steps": 7, "max_trials": 3},
    success_batch_size=8,
    extract_init_insights=True,
)

out = agent.generate(
    question=question,
    key=key,
    examples=SVAMP_FEWSHOT_EXAMPLES_REACT,
    prompt=EXPEL_REFLEXION_REACT_INSTRUCTION_SVAMP,
    reflect_examples=SVAMP_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    reflect_prompt=REFLEXION_REACT_REFLECT_INSTRUCTION_SVAMP,
    reflect_strategy="reflexion",
    additional_keys={},
    reflect_additional_keys={},
    use_dynamic_examples=True,
    extract_insights=True,
    patience=3,
    k_docs=24,
    num_fewshots=6,
    max_fewshot_tokens=1500,
    reranker_strategy=None,
    reset=False,
)

In [ ]:
out

### TabMWP

In [ ]:
question = """Read the following table regarding "Bowling Scores" and then write Python code to answer a question:

Name | Score
Amanda | 117
Sam | 236
Irma | 144
Mike | 164

Question: Some friends went bowling and kept track of their scores. How many more points did Mike score than Irma?"""
key = "20"


benchmark = "tabmwp"

reflexion_react_agent = ReflexionReAct(
    llm=llm,
    benchmark=benchmark,
    reflector=ReflexionReActReflector(llm=llm),
    # kwargs.
    max_reflections=3,
    max_trials=1,
    max_steps=6,
    max_tokens=5000,
    enc=tiktoken.encoding_for_model("gpt-3.5-turbo"),
)
agent = ExpeL(
    llm=llm,
    benchmark=benchmark,
    reflexion_react_agent=reflexion_react_agent,
    experience_memory=ExpeLExperienceMemory(
        experiences=[],
        strategy="task",
        embedder=HuggingFaceEmbeddings(),
        encoder=tiktoken.encoding_for_model("gpt-3.5-turbo")
    ),
    insight_memory=ExpeLInsightMemory(
        insights=[],
        max_num_insights=20,
        leeway=5
    ),
    # kwargs.
    reflexion_react_strategy_kwargs={"max_steps": 7, "max_trials": 3},
    success_batch_size=8,
    extract_init_insights=True,
)

out = agent.generate(
    question=question,
    key=key,
    examples=TABMWP_FEWSHOT_EXAMPLES_REACT,
    prompt=EXPEL_REFLEXION_REACT_INSTRUCTION_TABMWP,
    reflect_examples=TABMWP_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    reflect_prompt=REFLEXION_REACT_REFLECT_INSTRUCTION_TABMWP,
    reflect_strategy="reflexion",
    additional_keys={},
    reflect_additional_keys={},
    use_dynamic_examples=True,
    extract_insights=True,
    patience=3,
    k_docs=24,
    num_fewshots=6,
    max_fewshot_tokens=1500,
    reranker_strategy=None,
    reset=False,
)

In [ ]:
out

# Code

### HumanEval

In [ ]:
inst = {"task_id": "HumanEval/0", "prompt": "from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    \"\"\" Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    \"\"\"\n", "entry_point": "has_close_elements", "canonical_solution": "    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n", "test": "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\n    assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n\n"}
question = inst['prompt']
key = f"{inst['test']}\ncheck({inst['entry_point']})"


benchmark = "humaneval"

reflexion_react_agent = ReflexionReAct(
    llm=llm,
    benchmark=benchmark,
    reflector=ReflexionReActReflector(llm=llm),
    # kwargs.
    max_reflections=3,
    max_trials=1,
    max_steps=6,
    max_tokens=5000,
    enc=tiktoken.encoding_for_model("gpt-3.5-turbo"),
)
agent = ExpeL(
    llm=llm,
    benchmark=benchmark,
    reflexion_react_agent=reflexion_react_agent,
    experience_memory=ExpeLExperienceMemory(
        experiences=[],
        strategy="task",
        embedder=HuggingFaceEmbeddings(),
        encoder=tiktoken.encoding_for_model("gpt-3.5-turbo")
    ),
    insight_memory=ExpeLInsightMemory(
        insights=[],
        max_num_insights=20,
        leeway=5
    ),
    # kwargs.
    reflexion_react_strategy_kwargs={"max_steps": 7, "max_trials": 3},
    success_batch_size=8,
    extract_init_insights=True,
)

out = agent.generate(
    question=question,
    key=key,
    examples=HUMANEVAL_FEWSHOT_EXAMPLES_REACT,
    prompt=EXPEL_REFLEXION_REACT_INSTRUCTION_HUMANEVAL,
    reflect_examples=HUMANEVAL_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    reflect_prompt=REFLEXION_REACT_REFLECT_INSTRUCTION_HUMANEVAL,
    reflect_strategy="reflexion",
    additional_keys={},
    reflect_additional_keys={},
    use_dynamic_examples=True,
    extract_insights=True,
    patience=3,
    k_docs=24,
    num_fewshots=6,
    max_fewshot_tokens=1500,
    reranker_strategy=None,
    reset=False,
)

In [ ]:
out

### MBPP

In [ ]:
question = "Write a python function to find the first repeated character in a given string."
key = """assert first_repeated_char("abcabc") == "a"
assert first_repeated_char("abc") == None
assert first_repeated_char("123123") == "1\""""


benchmark = "mbpp"

reflexion_react_agent = ReflexionReAct(
    llm=llm,
    benchmark=benchmark,
    reflector=ReflexionReActReflector(llm=llm),
    # kwargs.
    max_reflections=3,
    max_trials=1,
    max_steps=6,
    max_tokens=5000,
    enc=tiktoken.encoding_for_model("gpt-3.5-turbo"),
)
agent = ExpeL(
    llm=llm,
    benchmark=benchmark,
    reflexion_react_agent=reflexion_react_agent,
    experience_memory=ExpeLExperienceMemory(
        experiences=[],
        strategy="task",
        embedder=HuggingFaceEmbeddings(),
        encoder=tiktoken.encoding_for_model("gpt-3.5-turbo")
    ),
    insight_memory=ExpeLInsightMemory(
        insights=[],
        max_num_insights=20,
        leeway=5
    ),
    # kwargs.
    reflexion_react_strategy_kwargs={"max_steps": 7, "max_trials": 3},
    success_batch_size=8,
    extract_init_insights=True,
)

out = agent.generate(
    question=question,
    key=key,
    examples=MBPP_FEWSHOT_EXAMPLES_REACT,
    prompt=EXPEL_REFLEXION_REACT_INSTRUCTION_MBPP,
    reflect_examples=MBPP_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    reflect_prompt=REFLEXION_REACT_REFLECT_INSTRUCTION_MBPP,
    reflect_strategy="reflexion",
    additional_keys={"tests": key},
    reflect_additional_keys={"tests": key},
    use_dynamic_examples=True,
    extract_insights=True,
    patience=3,
    k_docs=24,
    num_fewshots=6,
    max_fewshot_tokens=1500,
    reranker_strategy=None,
    reset=False,
)

In [ ]:
out